# 西太平洋副热带高压指数计算

副高指数分为面积指数，强度指数，脊线指数，西伸脊点四个部分。以刘芸芸的论文为标准，程序名为WPSH.py,采用python的命令行方式进行调用。程序完成后基于中国气象局监测中心的已有结果进行了对比验证，在文档中详细介绍了测试检验的过程和其中的各个注意点。监测中心的定义与标准（刘芸芸）存在部分出入，因此也一并给出相关定义。

## 0.程序执行

### 0.1 显示帮助信息

In [1]:
%run ../src/WPSH.py -h

usage: WPSH.py [-h] [-n NAME] [-v] hgt uwnd

positional arguments:
  hgt                   位势高度文件hgt
  uwnd                  水平风场文件uwind

optional arguments:
  -h, --help            show this help message and exit
  -n NAME, --name NAME  输出结果文件名
  -v, --verbose         输出详细信息


必须的参数有两个，位势高度文件hgt和水平风场文件uwind，-n,--name可以设置导出的结果文件名，结果以文本格式保存。

### 0.2 示例：计算1971年逐日ncep1副高指数

In [2]:
%run ../src/WPSH.py ../tmp/hgt.1971.nc ../tmp/uwnd.1971.nc --name ../tmp/SH.1971.CAL.txt

SH.1971.CAL.txt结果目录如下：
![](../tmp/wpsh1.png)
从左往右依次是年，月，日，面积指数，强度指数，脊线指数，西伸脊点。四个指数如果缺测则以－999代替。

## 1.面积指数

### 1.1 定义
1. 刘芸芸定义
> 在 10N 以北 110E - 180 范围内, 500 hPa 位势高度场上所有位势高度不小于 588 dagpm 的格点围成的面积总和.

2. 参考定义
> 500hPa高度场上，北半球10°N-60°N, 110°E-180°范围内≥588位势什米(dagpm)的网格点数称为西太平洋副高面积指数。

### 1.2 检验（1971年）

蓝线表示计算值，红点表示参考值，从图中可以看出面积指数的计算与参考值一致：
![](../tmp/wpsh_area.png)

## 2.强度指数

### 2.1 定义
1. 刘芸芸定义
> 在 10N 以北 110E - 180 范围内, 500 hPa 位势高度场上所有位势高度不小于 588 dagpm 的格点围成的面积与该格点高度值减去 587 dagpm 差值的乘积的总和.

2. 参考定义
> 500hPa高度场上，北半球10°N-60°N, 110°E -180°范围内≥588位势什米(dagpm)的网格高度值与587位势什米(dagpm)之差（向零取整）的累积值称为西太平洋副高强度指数。

### 2.2 检验（1971年）

蓝线表示计算值，红点表示参考值，从图中可以看出强度指数的计算与参考值一致：
![](../tmp/wpsh_int.png)

## 3.脊线指数

### 3.1 定义
1. 刘芸芸定义
> 在 10N 以北 110 - 150E 范围内, 588 dagpm 等值线所包围的副热带高压体内纬向风 u = 0 且 du/dy > 0 的特征线所在纬度位置的平均值; 若不存在 588 dagpm 等值线, 则定义 584 dagpm 等值线范围内纬向风 u = 0, 且 du/dy > 0 的特征线所在纬度位置的平均值; 若在某月不存在 584 dagpm 等值线, 则以该月的 1951 - 2010 年历史最小值代替.

2. 参考定义
> 500hPa高度场上，北半球10°N-60°N, 110°E-130°范围内逐条经线上副热带高压中心（即纬向风u=0、且du/dy>0）位置所在纬度的平均值称为印度副高脊线。

### 3.2 检验（1971年）

蓝点表示计算值，红点表示参考值，从图中可以看出脊线指数的结果虽然大致分布一致，但存在一定偏差，导致差别的原因有几个方面：
1. 网格分辨率不统一，计算数据是ncep1 2.5度的网格，而参考数据可能精度更高或者存在差值；
2. u=0:对于逐日资料，直接判断u==0的格点**极少**存在，因此程序中设定了一个误差常量UWND_ZERO_ERROR,u的绝对值小于该值则被判断为u=0,因此该值的大小会影响计算结果。经过多个值的实验，设在2左右能够最接近参考数据。
![](../tmp/wpsh_line1.png)
下图是（参考值－计算值）的差对时间做的图：
![](../tmp/wpsh_line2.png)

## 4.西伸脊点

### 4.1 定义
1. 刘芸芸定义
> 在 90E - 180 范围内, 588 dagpm 最西格点所在的经度. 若在 90E 以西则统一记为 90E, 若在某月不存在 588 dagpm 等值线, 则以该月的 1951 - 2010 年历史最大值代替.

2. 参考定义
> 500hPa高度场上，北半球90°E -180°范围内588位势什米等值线最西端位置所在经度称为西太平洋副高西伸脊点。

### 4.2 检验（1971年）

蓝点表示计算值，红点表示参考值，从图中可以看出西伸脊点的结果虽然大致分布一致，但存在一定偏差，导致差别的原因有几个方面：
1. 网格分辨率不统一，计算数据是ncep1 2.5度的网格，而参考数据可能精度更高或者存在差值；
2. 588等值线的判断:对于粗网格的ncep1资料在判断588dagpm的副高体等值线时判断条件的细微变化会对结果产生很大的影响，因此程序中设定了一个误差常量RIDGE_POINT_ERROR,hgt在[5880-RIDGE_POINT_ERROR,5880+RIDGE_POINT_ERROR]（数据中值的单位是0.1dagpm）都被认为是588等值线上的点。经过多个值的实验，设在1.5左右能够最接近参考数据。
![](../tmp/wpsh_points2.png)
下图是（参考值－计算值）的差对时间做的图：
![](../tmp/wpsh_points1.png)